In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("project.ipynb")

# Project 2 – COVID Vaccinations 🦠

## DSC 80, Spring 2022

### Checkpoint Due Date: Thursday, April 21st (Questions 1, 2, 6, 8, and 10)
### Due Date: Saturday, April 30th

## Instructions 📝

Welcome to Project 2!

---

### Working on the Project 💻

This Jupyter Notebook contains the statements of the problems and provides code and Markdown cells to display your answers to the problems.  

* Like in labs, your coding work will be developed in the accompanying `project.py` file, that will be imported into the current notebook. This code will be autograded.
* Note that there is no manually-graded component to Project 2, so the only thing you will ever submit is `project.py`.
* **For the checkpoint, you only need to turn in a `project.py` containing solutions for Questions 1, 2, 6, 8, and 10!**
    - The "Project 2 Checkpoint" autograder on Gradescope does not thoroughly check your code – it only runs the public notebook tests on the checkpoint questions to make sure that you have completed them. When you submit the final version of the project, we will use hidden tests to check your answers more thoroughly.
    - Note that this means you will ultimately have to submit the project twice – once to the "Project 2 Checkpoint" autograder (Questions 1, 2, 6, 8, and 10 only), and once to the "Project 2" autograder (once you're fully done).
    - The reason for skipping around is that **the project has two distinct parts, and you should start on both parts by the time the checkpoint is due**. Note that the checkpoint questions are less than half of the total project (and some of them are easier than the average project question), so budget your time appropriately for completing the rest of the project.

**Do not change the function names in the `project.py` file!**
- The functions in the `project.py` file are how your assignment is graded, and they are graded by their name.
- If you changed something you weren't supposed to, just use git to revert! Ask us if you need help with this, or Google around for `git revert`.

**Tips for developing in the `project.py` file**:
- Do not change the function names in the starter code; grading is done using these function names.
- Do not change the docstrings in the functions. These are there to tell you if your work is on the right track!
- You are **encouraged to write your own additional functions** to solve the questions! 
- Always document your code!

**Tips for testing the correctness of your answers:**

Once you write your code in `project.py`, you should use your notebook to inspect and analyze your functions' outputs for correctness. You should also run the doctests on the command line, and submit a version of your project to the Gradescope autograder well in advance of the deadline to ensure that there are no unexpected bugs in your code or your computing environment.

<span style='color:red'><b>Note that in this project, the doctests and public Otter notebook tests are not identical</b></span>, though they test similar ideas. In particular, the doctests often test your functions on smaller subsets of the larger datasets that we use in the notebook. **You should regularly run both.**

### Warning! ⚠️

This project contains larger datasets than usual. They aren't huge, but they aren't tiny. The autograder has been set to allow more memory usage than usual, but if you write inefficient code you may exhaust all of the allocated memory, causing the autograder to fail. If this happens, the autograder will say that your submission was formatted incorrectly.

**Writing efficient code is part of the project!** Be sure to submit your code to the Gradescope autograder regularly to check that it is efficient enough. If your code fails to run on the autograder due to efficiency reasons and it is past the late deadline, we will perform a "catastrophic regrade" by removing your inefficient code (deleting that function definition) and charging the usual cost of two slip days. This is expensive, so be sure to check that your code works with the autograder with plenty of time before the deadline!

### Working with a Partner 🧑‍🤝‍🧑

You are allowed to work with a partner on projects in DSC 80. If you do work with a partner, you must follow the [Pair Programming Guidelines](https://dsc10.com/pair-programming/). Specifically, you must be actively working on the project at the same time on one computer. Splitting up the project and working on it separately **is not** pair programming.

You can use [this sheet](https://docs.google.com/spreadsheets/d/1PMtGpd4U6rYBn6Ut6eHQzSo4PdBwluU-ppx87ROy_N8/edit?usp=sharing) to find a partner. Your partner does not have to be in the same lecture section as you.

Note that if you do work with a partner, you and your partner must submit the Checkpoint together and the whole project together.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
import os

import pathlib
import urllib.request
from ipywidgets import interact, FloatSlider

import matplotlib.pyplot as plt
plt.style.use('seaborn-white')
plt.rc('figure', dpi=100, figsize=(8, 5))
plt.rc('font', size=12)

In [ ]:
from project import *

## About the Assignment

Data science has been an important tool in understanding the development of the COVID-19 pandemic. In this project, we'll take a took at two real-world datasets that measure vaccination rates and effectiveness.

---

### Navigating the Project

Click on the links below to navigate to different parts of the project. Note that Questions 1, 2, 6, 8, and 10 constitute your Checkpoint submission.

- [Part 1: Global Vaccination Rates 🌎](#part1)
    - [Question 1 (Checkpoint Question)](#Question-1-(Checkpoint-Question))
    - [Question 2 (Checkpoint Question)](#Question-2-(Checkpoint-Question))
    - [Question 3](#Question-3)
    - [Question 4](#Question-4)
    - [Question 5](#Question-5)
- [Part 2: Vaccine Effectiveness 💉](#part2)
    - [Question 6 (Checkpoint Question)](#Question-6-(Checkpoint-Question))
    - [Question 7](#Question-7)
    - [Question 8 (Checkpoint Question)](#Question-8-(Checkpoint-Question))
    - [Question 9](#Question-9)
    - [Question 10 (Checkpoint Question)](#Question-10-(Checkpoint-Question))
    - [Question 11](#Question-11)

---

The file `data/covid_vaccinations_updated.csv` contains information on the number of COVID vaccinations that have been administered in various countries and regions of the world, as of April 7th, 2022:

In [ ]:
vacs = pd.read_csv(os.path.join('data', 'covid_vaccinations_updated.csv'))
vacs

<a name='part1'></a>

## Part 1: Global Vaccination Rates 🌎

One question we might hope to answer with this data is, **which countries have the highest vaccination rates?** Unfortunately, the messiness of real-world data means that answering even this simple question requires a bit of cleaning and organization. We'll do this cleaning and organization in the next few questions.

First, we need to find the total number of vaccinated individuals in each country/region. To do this, we have to understand what the values in the `'Doses_admin'` column mean. Are they the number of doses administered on *that day*? Or are they the *cumulative* number of doses administered? This can be answered with a plot.

In [ ]:
vacs[vacs['Country_Region'] == 'US'].plot(x='Date', y='Doses_admin')
plt.title('Doses_admin in the US')
plt.xticks(rotation=45);

Since the curve is always increasing, this plot suggests that the `'Doses_admin'` column contains the *cumulative* number of doses administered, and this is indeed what the column is *supposed* to be. However, you shouldn't *assume* that the reported vaccination rates are actually cumulative. For example, let's look at the plot for Afghanistan:

In [ ]:
vacs[vacs['Country_Region'] == 'Afghanistan'].plot(x='Date', y='Doses_admin')
plt.title('Doses_admin in Afghanistan')
plt.xticks(rotation=45);

Notice the dips in the graph? Unless there were *negative* doses administered (which would be awkward), we must conclude that the data is messy - there will be instances where, either due to errors in reporting or in recording, the cumulative number of doses actually *decreases* from one day to the next.

### Question 1 (Checkpoint Question)

Let's explore whether the messiness seen above is common.

#### `is_monotonic`

Create a function named `is_monotonic` which accepts a 1D `numpy` array and returns `True` if every entry in the array (except the first) is greater than or equal to the previous entry, and `False` otherwise.

***Hint***: There's a `numpy` function for computing the difference between consecutive elements of an array.

<br>

#### `monotonic_by_country`

Create a function named `monotonic_by_country` which accepts a DataFrame like `vacs` and returns a DataFrame with one row for each country and three `bool` columns – `'Doses_admin_monotonic'`, `'People_partially_vaccinated_monotonic'`, and `'People_fully_vaccinated_monotonic'`. An entry in the `'Doses_admin'` column should be `True` if the country's `'Doses_admin'` is monotonically increasing and `False` otherwise; likewise for the other columns. The index of the returned DataFrame should contain country names.

***Note:*** No looping is allowed, but `groupby` is your friend. Use the `is_monotonic` function, along with the assumption that the input DataFrame `vacs` is already sorted by date.

In [ ]:
# run this to see the result, and don't change this cell -- it is used by the tests
mono = monotonic_by_country(vacs)
mono

In [ ]:
grader.check("q1")

If you've completed Question 1 correctly, you should see that it is not rare for the data to violate the monotonicity assumption. We'll have to take this into account when doing our data analysis.

### Question 2 (Checkpoint Question)

We'd like to get the total number of doses administered, people partially vaccinated, and people fully vaccinated for each country. If the data were truly monotonic, we could simply group by country and find the maximum within each group and within each column – this would be the overall cumulative total. However, if the data has errors that make it not monotonically increasing, there could be a noisy "spike" in the graph, so that the maximum may be an unfaithful measure of the total number of doses. Instead of the maximum, we should use something more robust, such as **the 97th percentile**.

Create a function named `robust_totals` that accepts a DataFrame like `vacs` and returns a DataFrame with one row for each country and three columns – `'Doses_admin'`, `'People_partially_vaccinated'`, and `'People_fully_vaccinated'`. The `'Doses_admin'` column should contain the 97th percentile of the `'Doses_admin'` values for each country; likewise for the other columns. The index of the returned DataFrame should contain country names.

In [ ]:
# run this to see the result, and don't change this cell -- it is used by the tests
tots = robust_totals(vacs)
tots

In [ ]:
grader.check("q2")

We'd like to compute for each country the number of people vaccinated as a percentage of the total population. To this end, we have downloaded a CSV file containing the population of every country in the world:

In [ ]:
pops_raw = pd.read_csv(os.path.join('data', 'populations.csv'))
pops_raw

### Question 3

Unfortunately, this DataFrame's data types require some cleaning. All of the columns except the first are numeric and should either be integers or floats, as appropriate, but many of them are being stored as strings. Additionally, some rows have missing values.

Create a function named `fix_dtypes` which accepts a DataFrame like `pops_raw` above and returns a DataFrame with exactly the same columns and rows, but with the data types "fixed" to be appropriate for the data contained within. In addition, ensure that all missing values are represented by `np.NaN`. All percentages should be represented as decimals – e.g., 27% should be 0.27.

***Hint:*** You can make helper functions! It's OK to loop over the **columns** of the DataFrame (but not the rows). And remember that `np.NaN` is represented as a float, so even if the values in a column look like integers, if the column contains any `np.NaN`s then the `dtype` must be float.

As always, make sure that your function does not change the DataFrame it is called on!

In [ ]:
# run this to see the result, and don't change this cell -- it is used by the tests
pops = fix_dtypes(pops_raw)
pops

In [ ]:
grader.check("q3")

### Question 4

Remember that our goal is to compute the percentage of people vaccinated in each country. Our next step is to take the population numbers from the previous problem and insert them into the DataFrame of vaccination numbers (`tots`). In theory this is a simple `concat` or `merge`, but there's a problem: the names of the countries are not always the same in each DataFrame.

#### `missing_in_pops`
First, create a function named `missing_in_pops` which takes in two DataFrames, the first, like `tots` above, containing the total number of vaccinations per country, and the second like `pops` above, containing the population of each country. It should return a Python **set** of names that appear in `tots` but not in `pops`.

***Hint:*** `pandas` indexes have a method that checks if each element of the index is a member of some other collection. **Do not** use a `for`-loop.

<br>

#### `fix_names`
After running `missing_in_pops`, you'll notice that there are 14 countries that appear in `tots` that do not appear in `pops`. It turns out that most of these countries *do* appear in `pops`, just under a different name or spelling. Using the power of Google or your domain knowledge, combined with an exploration of the `tots` DataFrame, create a function named `fix_names` that accepts one argument – a DataFrame like `pops` – and returns a copy of `pops`, but with the `'Country (or dependency)'` column changed so that all countries that appear in `tots` also appear in the result, with a few exceptions listed below. 

For example, the country `'Burma'` appears in `tots`, but is listed as `'Myanmar'` in `pops`. In the DataFrame returned by your function, `'Myanmar'` should be changed to `'Burma'` to match `tots`.

As noted above, there are a few exceptions:

- `'Kosovo'` does not appear in `pops`. You do not need to change an entry in `pops` to match it.
- There is a region simply called `'World'`. This doesn't appear in `pops`. You do not need to change an entry in `pops` to match it.
- There is both `'US'` and `'US (Aggregate)` in `tots` but neither appear in `pops`. You only need to match `'US'`, and can leave `'US (Aggregate)'` unmatched.

In [ ]:
# run this to see the result, and don't change this cell -- it is used by the tests
missing = missing_in_pops(tots, pops)
missing

In [ ]:
# run this to see the result, and don't change this cell -- it is used by the tests
pops_fixed = fix_names(pops)
pops_fixed

In [ ]:
grader.check("q4")

### Question 5

Create a function named `top_k_fully_vaccinated` that accepts three arguments: a DataFrame like `tots`, a DataFrame like `pops_fixed`, and an integer, `k`, and returns a Series of the $k$ top vaccination rates of any country, sorted in descending order. For the purposes of this question, we define vaccination rate to be the number of fully vaccinated individuals divided by the total population. 

The index of the Series should be the country name, and the rates should be decimal numbers between 0 and 1 (you may have to clip in order to ensure this is the case). Don't worry about ties.

In [ ]:
# run this to see the result, and don't change this cell -- it is used by the tests
top_k = top_k_fully_vaccinated(tots, pops_fixed, 10)
top_k

In [ ]:
grader.check("q5")

That's it for Part 1 of the project!

---

<a name='part2'></a>

## Part 2: Vaccine Effectiveness 💉

There has naturally been much interest in the effectiveness of COVID-19 vaccines. In particular, recent reports suggest that the effectiveness of vaccines – and the Pfizer vaccine in particular – may decrease over time. While this is true, the public perception of declining vaccine effectiveness has been affected by what is often a poor use of statistics by media outlets.

For example, take the following blurb from [this article](https://www.cbc.ca/news/world/israel-covid-delta-variant-booster-1.6159472) on the Canadian Broadcasting Corporation's website (emphasis added):

> Meanwhile, Israeli health officials reported what appeared to be a waning efficacy of the vaccine, including among those who had been double vaccinated. **Data showed that of the serious cases being admitted to hospital, around 60 per cent of patients were people who had been fully vaccinated**, though most were over 60 or with underlying health conditions.

This stat sounds terrible – if you're sick in a hospital bed with COVID-19, you're actually more likely to be vaccinated than not! If you're not a data scientist, you might come away with the conclusion that the vaccine is no longer effective against serious illness, or – even worse – that getting the vaccine will **increase** your risk of sickness. Of course, this statistic by itself is actually meaningless. For instance, imagine a world in which 100% of people are vaccinated. Then 100% of people admitted to the hospital will be vaccinated, too! In Question 8, we will explain the theory behind **vaccine effectiveness**, a statistic that more meaningfully measures the improvement in outcomes for vaccinated individuals over unvaccinated individuals.

To start, let's load in some data. For this project, we'll work with a dataset of vaccine effectiveness in Israel through mid-summer 2021. The vaccination and hospitalization records in the dataset are real. Israel was mostly vaccinated using the Pfizer vaccine, and was one of the first places where lots of data on the Delta variant was gathered.

In [ ]:
# Download Israeli COVID vaccinations data from the ☁️
if not pathlib.Path(os.path.join('data', 'israel.csv')).exists():
    urllib.request.urlretrieve(
        'https://f000.backblazeb2.com/file/dsc-data/covid-israel/israel.csv',
        os.path.join('data', 'israel.csv')
    )

In [ ]:
israel_raw = pd.read_csv(os.path.join('data', 'israel.csv'))
israel_raw

Each row in the dataset is a person. Some are vaccinated (marked with a 1 in the "Vaccinated" column), and some became severely ill (marked with a 1 in the "Severe Sickness" column).

As always, we will first check that the data is clean.

In [ ]:
israel_raw.info()

### Question 6 (Checkpoint Question)

If you look at the data types of each column in `israel_raw`, you'll see that the `'Age'` column has a `dtype` of `object`. If you investigate further, you will see that a special value has been used to indicate that some ages are missing. Write a function named `clean_israel_data` that accepts a DataFrame like `israel_raw` and returns a *new* DataFrame where the missing ages are replaced by `np.NaN`s and the `'Age'` column's data type is `float`. Furthermore, the `'Vaccinated'` and `'Severe Sickness'` columns should be stored as `bool`s. The shape of the returned DataFrame should be the same as `israel_raw`, and, as usual, your function should not modify the input DataFrame.

In [ ]:
# run this to see the result, and don't change this cell -- it is used by the tests
israel = clean_israel_data(israel_raw)
israel

In [ ]:
grader.check("q6")

We now need to decide what to do with the missing values in the `'Age'` column. Can we simply drop those rows? First, let's look at whether the missing values might be MCAR, NMAR, missing by design, etc. We'll first check to see if the distribution of the other columns differs depending on whether the ages are missing or not:

In [ ]:
israel.assign(null_age=israel['Age'].isna()).groupby('null_age').mean()

### Question 7

Because the vaccination rate and severe sickness rate appear to be about the same, whether or not `'Age'` is missing, we hypothesize that `'Age'` is MCAR (missing completely at random). Let's investigate.

#### `mcar_permutation_tests`

Write a function named `mcar_permutation_tests` that accepts two arguments – a DataFrame like `israel` and a number `n_permutations` of permutations – and runs the two permutation tests described below. Your function should return a 2-tuple where the first entry is an array of the simulated test statistics for the first permutation test, and the second entry is an array of simulated test statistics for the second permutation test.

- The first permutation test should check the null hypothesis that values in the `'Vaccinated'` column for rows where the `'Age'` was missing were drawn from the same distribution as the values in rows where `'Age'` was not missing. The alternative hypothesis should be that they were drawn from *different* distributions. As your test statistic, use the **absolute difference in group means**. 
- The second permutation test should do the same, but for the `'Severe Sickness'` column.

***Note:*** Your code should run in less than two minutes when called with `n_permutations=100`. You should be able to run both permutation tests in a single `for`-loop.

<br>

#### `missingness_type`

Then, create a function named `missingness_type` that takes no arguments and returns a single integer corresponding to the option below that you think describes the type of missingess in this data:

1. MCAR (Missing completely at random)
2. MAR (Missing at random)
3. NMAR (Not missing at random)
4. Missing by design

In [ ]:
# run this to see the result, and don't change this cell -- it is used by the tests
test_stats_vax, test_stats_sick = mcar_permutation_tests(israel, 100)

In [ ]:
grader.check("q7")

Let's proceed by simply dropping the rows with missing values:

In [ ]:
vax = israel.dropna()

Moving forward, use `vax`, not `israel`.

### Question 8 (Checkpoint Question)

***Note:*** Assume that "hospitalization" and "severe illness" mean the same thing; that is, if someone is hospitalized they developed severe illness and vice versa.

As mentioned at the start of Part 2, looking at the probability of being vaccinated if you're hospitalized is meaningless, since if everyone is vaccinated, this probability will be 1. Instead, we care about **the probability of being hospitalized if you're vaccinated.** More than that, what we care most about is how this probability relates to the probability of being hospitalized if you're **unvaccinated**.

To put it more concretely, let's say that 1 in 100 vaccinated people are hospitalized, while 10 in 100 unvaccinated people are hospitalized. This means that unvaccinated people are 10x more likely to be hospitalized than vaccinated people. To put it another way, if those 10 hospitalized unvaccinated people had been vaccinated, we'd expect that 9 of them would not have been hospitalized. That is, the vaccine would have prevented 90% of the hospitalizations, and we might therefore say that it is 90% effective against severe illness.

This is the intuition behind the definition of **vaccine effectiveness** (see the CDC's [page](https://www.cdc.gov/csels/dsepd/ss1978/lesson3/section6.html) for more information). That is, to compute a vaccine's effectiveness, we need to know two things:

1. The proportion of vaccinated people who developed severe illness (i.e., the probability that a vaccinated person will be hospitalized). Call this $p_V$.
2. The proportion of unvaccinated people who developed severe illness (i.e., the probability that an unvaccinated person will be hospitalized). Call this $p_U$.

Then:

$$
\text{effectiveness} = 1 - \frac{p_V}{p_U} = \frac{p_U - p_V}{p_U}
$$

**What this formula calculates, in words, is the proportion of unvaccinated people with severe illness who would (we expect) not have severe illness if they were vaccinated.** For instance, if no one who is vaccinated becomes severely ill, then $p_V = 0$ and the effectiveness is 1 (100%). On the other hand, if the vaccine is no better than being unvaccinated, $p_V = p_U$ and the effectiveness is 0 (0%).


Many news articles report effectiveness numbers, and this is certainly better than reporting the percentage of hospital patients that are vaccinated. Note, though, that there are different types of effectiveness – effectiveness against severe illness, effectiveness against symptomatic illness, effectiveness against death – and articles often are vague about which they are using. But even then, effectiveness can *still* be misleading due to statistical quirks, as we'll now see.


<br>

**Your job is to** create a function named `effectiveness` that accepts a DataFrame like `vax` and returns the effectiveness of the vaccine against severe illness as a proportion between 0 and 1.

***Note:*** Assume that you will not run into any division-by-zero errors.

In [ ]:
# run this to see the result, and don't change this cell -- it is used by the tests
eff_overall = effectiveness(vax)
eff_overall

In [ ]:
grader.check("q8")

If you implemented `effectiveness` correctly, you should see a vaccine effectiveness between 60% and 70%. This is OK, but not great! It means that the vaccine is only around 60-70% effective at preventing severe illness overall.

But remember Simpson's Paradox, which roughly says that sometimes a result that appears in aggregated data disappears when we look at disaggregated data. So let's disaggregate the data and perform the same calculation. That is, let's compute the effectiveness of the vaccine within each of several age groups. For convenience, here is a list of the age groups we'll consider:

In [ ]:
AGE_GROUPS = [
    '12-15',
    '16-19',
    '20-29',
    '30-39',
    '40-49',
    '50-59',
    '60-69',
    '70-79',
    '80-89',
    '90-'
]

Each string above lists an age group. For example, the age group `'12-15'` should contain anyone aged 12, 13, 14, or 15. The last age group, '90-', denotes everyone who is 90 or above. `vax` does not contain anyone younger than 12.

### Question 9

Create a function named `stratified_effectiveness` that accepts one argument – a DataFrame like `vax` – and returns the effectiveness of the vaccine within each of the age groups in `AGE_GROUPS`. The return value of the function should be a Series of the same length as `AGE_GROUPS`, with the index of the Series being age groups as strings.

In [ ]:
# run this to see the result, and don't change this cell -- it is used by the tests
s_eff = stratified_effectiveness(vax)
s_eff

In [ ]:
grader.check("q9")

If done correctly, you should notice that the effectiveness of the vaccine within each age group is higher than the overall effectiveness. In fact, the effectiveness of every age group is above 80%, and for most age groups it is above 90%.

This is an instance of [Simpson's paradox](https://dsc80.com/resources/lectures/lec08/lec08.html#Simpson's-paradox). This result might seem strange, or even impossible – how can *every one* of the age groups see a higher effectiveness rate than the overall effectiveness rate? We'll take a deeper look at this in the next question, but for now let's consider a slightly different scenario where we can give dosages of a vaccine on a scale, so that some people can receive more vaccine than others. We expect that the higher a dosage someone receives, the less sick they become. But suppose we plot the data and we see something like this:

<img src="./data/simpsons-aggregated.svg">

It actually looks like the more dosage someone receives, the more sick they get! But suppose we were to *disaggregate* the data into two age groups – old and young. That is, let's look at the same plot, but now mark young people as red and older people as purple. We might see this:

<img src="./data/simpsons.svg">

Now we see a different story: the larger the dosage, the less the sickness within each age group. Therefore, while the overall trend is a *positive* relationship between dosage and sickness, the within-group trends are all negative, as we'd hope. **The reason for this difference comes from the fact that, for each dosage level, the sickness rate of older people is greater than the sickness rate for younger people.**

If our data gives different answers depending on how we aggregate it, which answer should we use? In this case, we don't care about overall effectiveness – we care about how effective the vaccine will be for an *individual*. The data shows that, no matter what your age, unvaccinated people are ten times as likely to develop severe illness than vaccinated people, and it shows that the vaccine's effectiveness against severe illness is still quite strong.

The plots above may help to understand Simpson's paradox a little bit more, but it may still be unclear how each age group's effectiveness rate can be higher than the overall effectiveness rate. To explore this in more depth, **we'll build a calculator that takes in several parameters and calculates three effectivenesses: the overall (aggregate) effectiveness, the effectiveness among young people, and the effectiveness among old people.** For simplicity, we'll assume that there are just as many old people as young people.

To calculate these effectivenesses, you need to know six numbers:

1. The proportion of young people that are vaccinated.
2. The proportion of old people that are vaccinated.
3. The probability that a young vaccinated person will be hospitalized.
4. The probability that a young *un*vaccinated person will be hospitalized.
5. The probability that an old vaccinated person will be hospitalized.
6. The probability that an old *un*vaccinated person will be hospitalized.

Given these six pieces of information, you can calculate the overall effectiveness, the effectiveness in young people, and the effectiveness in old people.

How, you may ask? Well, to find the effectiveness in young people, you can use the effectiveness definition from [Question 8](#Question-8-(Checkpoint-Question)) ($1 - \frac{p_V}{p_U}$) directly since you are given the probability that a young vaccinated person will be hospitalized ($p_V$, for young people) and the probability that a young unvaccinated person will be hospitalized ($p_U$, for young people). The same applies for the effectiveness in old people.

To find the overall effectiveness, you need to find the values of $p_V$ and $p_U$ in the overall population, given  the six parameters that `effectiveness_calculator` takes in. 
- One approach is to express all six parameters in terms of conditional probability notation and to use your knowledge of probability theory from DSC 40A. One idea to look at in particular is the law of total probability ([Wikipedia article](https://en.wikipedia.org/wiki/Law_of_total_probability), [slides from DSC 40A](https://dsc-courses.github.io/dsc40a-2021-fa/resources/lecture/lec14-filled.pdf)).
- Another approach is to fix concrete numbers. 
    - For instance, assume that there are 1000 old people and 1000 young people (recall, we are operating under the assumption there are an equal number of old and young people, for simplicity).
    - To calculate the probability that a vaccinated person is hospitalized, you need to know the _number_ of vaccinated people that are hospitalized in this hypothetical example. If you're told that 20% of young people are vaccinated and that 10% of young vaccinated people will be hospitalized, you know that $1000 \cdot 0.2 \cdot 0.1 = 20$ young vaccinated people will be hospitalized. 
    - If you're also told that 60% of old people are vaccinated and that 30% of old vaccinated people will be hospitalized, then you know that $1000 \cdot 0.6 \cdot 0.3 = 180$ old vaccinated people will be hospitalized.
    - This means that 200 of 800 ($1000 \cdot 0.2 + 1000 \cdot 0.6$) vaccinated people will be hospitalized, so the overall $p_V$ is $p_V = \frac{200}{800} = 0.25$ in this example.
    - You can do the same for unvaccinated people to find the overall $p_U$.
    - Working with concrete numbers like this *is* using probability theory, just with extra steps. But it might be less abstract and easier to reason about.

### Question 10 (Checkpoint Question)

Create a function named `effectiveness_calculator` that accepts 6 arguments:

1. `young_vaccinated_prop`: The proportion of young people that are vaccinated.
2. `old_vaccinated_prop`: The proportion of old people that are vaccinated.
3. `young_risk_vaccinated`: The probability that a young vaccinated person will be hospitalized.
4. `young_risk_unvaccinated`: The probability that a young *un*vaccinated person will be hospitalized.
5. `old_risk_vaccinated`: The probability that an old vaccinated person will be hospitalized.
6. `old_risk_unvaccinated`: The probability that an old *un*vaccinated person will be hospitalized.

It should return a dictionary with three keys: `'Overall'`, `'Young'`, and `'Old'`, whose values are the overall effectiveness, the effectiveness within young people, and the effectiveness within old people, respectively. This will allow you to experiment with Simpson's paradox by plugging in different values and seeing what happens.

***Note:*** All arguments to `effectiveness_calculator` are numbers between 0 and 1 (inclusive).

***Hint:*** You are given two example inputs to `effectiveness_calculator` – one in the doctest and one in the notebook. Make sure your code works correctly for **both** of them. 

In [ ]:
# run this to see the result, and don't change this cell -- it is used by the tests
eff_example = effectiveness_calculator(
    young_vaccinated_prop=0.01,
    old_vaccinated_prop=0.99,
    young_risk_vaccinated=0.01,
    young_risk_unvaccinated=0.20,
    old_risk_vaccinated=0.10,
    old_risk_unvaccinated=0.50
)
eff_example

In [ ]:
grader.check("q10")

It might be helpful to think about the result of your effectiveness calculator. In the example above, we have constructed an extreme situation where essentially all older people are vaccinated, but almost no younger people are. Therefore, when calculating the overall vaccine effectiveness, we're using the probability of illness given vaccination – which is essentially the probability of severe illness if you're old – and comparing it to the probability of severe illness if you're unvaccinated, which is essentially the probability of severe illness if you're young.

In fact, it is possible to create an example where the overall effectiveness is negative (taking the vaccine is worse than not taking it!), but where the effectiveness within groups is above 80%. The next question asks you to do just this. 

To help you with your experimentation, we've created an animated version of your `effectiveness_calculator`, that allows you to test different arguments by dragging sliders. Run the next cell to use it.

In [ ]:
interact(effectiveness_calculator,
         young_vaccinated_prop=FloatSlider(min=0.01, max=0.99, step=0.01, value=0.01),
         old_vaccinated_prop=FloatSlider(min=0.01, max=0.99, step=0.01, value=0.99),
         young_risk_vaccinated=FloatSlider(min=0.01, max=0.99, step=0.01, value=0.01),
         young_risk_unvaccinated=FloatSlider(min=0.01, max=0.99, step=0.01, value=0.20),
         old_risk_vaccinated=FloatSlider(min=0.01, max=0.99, step=0.01, value=0.10),
         old_risk_unvaccinated=FloatSlider(min=0.01, max=0.99, step=0.01, value=0.50));

### Question 11

Create a function named `extreme_example` that accepts no arguments and returns a dictionary whose keys are the arguments to the function `effectiveness_calculator`. When your function is called and the dictionary is passed to `effectiveness_calculator`, it should return an `'Overall'` effectiveness that is negative and `'Young'` and `'Old'` effectivenesses that are both over 0.8.

***Note:*** Use the animated calculator above to help you, but **make sure to think about what the numbers mean**. There are many possible correct answers!

In [ ]:
# run this to see the result, and don't change this cell -- it is used by the tests
eff_extreme = effectiveness_calculator(**extreme_example())
eff_extreme

In [ ]:
grader.check("q11")

## Congratulations, you've finished Project 2! 🎉

Submit your `project.py` file to Gradescope. Note that you only need to submit the `project.py` file; this notebook should not be uploaded because there are no manually-graded questions in this project.

Before submitting, you should ensure that all of your work is in the `project.py` file. You can do this by running the doctests below, which will verify that your work passes the public tests **and** that your work is in the `project.py` file. Run the cell below; you should see no output.

In [ ]:
!python -m doctest project.py

In addition, `grader.check_all()` will verify that your work passes the public tests. Ultimately, the Gradescope autograder is also going to run `grader.check_all()`, so you should ensure these pass as well (which they should if the doctests above passed).

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [ ]:
grader.check_all()